In [ ]:

from collections import defaultdict
from pathlib import Path
from typing import List, Tuple, Dict, Iterator, Callable

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from matplotlib.gridspec import GridSpec

In [ ]:
logs_path = Path('../data/saved/logs')
save_path = Path('images/results')
save_path.mkdir(parents=True, exist_ok=True)
sns.set_theme()
sns.set_palette('muted')

In [ ]:
def get_log_identifier(log_path: Path) -> int:
    filename = log_path.stem
    identifier = filename.split('-')[-1]
    return int(identifier)


def iter_over_log_files(path_to_logs: Path) -> Iterator[str]:
    for log_file in sorted(path_to_logs.glob('*-log-epoch-*.log'), key=get_log_identifier):
        yield log_file


def parse_training_loss(log_path: Path, activate_special: Callable[[str], bool], special_mode: bool) -> \
        Tuple[List[float], List[int], List[float], List[int], Dict[str, List[float]], float, int, Dict[str, float],
              bool]:
    steps_list = []
    loss_list = []
    special_steps = []
    special_loss = []
    val_loss = None
    val_step = None
    train_scores = defaultdict(list)
    val_scores = {}
    epoch = 0
    iteration_max = 0
    with open(log_path, 'r') as log_file:
        for line in log_file:
            if activate_special(line):
                special_mode = True
                continue

            if 'ROUGE' in line or 'F1' in line:
                line_split = line.split(', ')
                if 'memory' in line: # Training scores
                    for score in line_split[-4:-1]:
                        score, value = score.split(': ')
                        train_scores[score].append(float(value))
                else: # Validation scores
                    loss = line_split[2]
                    val_loss = float(loss.replace('Loss: ', ''))
                    val_step = (epoch + 1) * iteration_max
                    for score in line_split[-3:]:
                        score, value = score.split(': ')
                        val_scores[score] = float(value)
                    continue

            if 'Loss' not in line:
                continue
            line_split = line.split(',')
            step = line_split[0]
            epoch, iteration = step.split(' Iter: ')
            epoch = int(epoch.replace('Epoch: ', ''))
            iteration, iteration_max = iteration.split('/')
            iteration = int(iteration)
            iteration_max = int(iteration_max)
            loss = line_split[2].replace('Loss: ', '')
            loss = float(loss)
            if special_mode:
                special_loss.append(loss)
                special_steps.append(epoch * iteration_max + iteration)
            else:
                loss_list.append(loss)
                steps_list.append(epoch * iteration_max + iteration)

    return loss_list, steps_list, special_loss, special_steps, train_scores, val_loss, val_step, val_scores, \
           special_mode


def parse_full_validation_summarization(log_path: Path) -> Tuple[List[float], Dict[str, List[float]]]:
    scores = defaultdict(list)
    loss_list = []
    with open(log_path / f'{log_path.stem}-log-full_eval.log', 'r') as val_file:
        for line in val_file:
            if 'test phase' in line:
                break

            if 'ROUGE' not in line:
                continue

            line_split = line.split(', ')
            loss = line_split[2]
            val_loss = float(loss.replace('Loss: ', ''))
            loss_list.append(val_loss)
            for score in line_split[-3:]:
                score, value = score.split(': ')
                scores[score].append(float(value))

    return loss_list, scores


def get_model_performance(path: Path, activate_special: Callable[[str], bool] = lambda _: False,
                                  use_full_validation: bool = False) -> \
        Tuple[List[float], List[int], List[float], List[int], Dict[str, List[float]], List[float], List[int],
              Dict[str, List[float]]]:
    special_mode = False
    loss_all = []
    steps_all = []
    special_loss_all = []
    special_steps_all = []
    val_loss_all = []
    val_steps_all = []
    train_scores_all = defaultdict(list)
    val_scores_all = defaultdict(list)
    for log_file in iter_over_log_files(path):
        loss, steps, special_loss, special_steps, train_scores, val_loss, val_steps, val_scores, special_mode = \
            parse_training_loss(log_file, activate_special, special_mode)
        loss_all += loss
        steps_all += steps
        special_loss_all += special_loss
        special_steps_all += special_steps
        for score, value in train_scores.items():
            train_scores_all[score] += value
        val_loss_all.append(val_loss)
        val_steps_all.append(val_steps)
        for score, value in val_scores.items():
            val_scores_all[score].append(value)

    if use_full_validation:
        val_loss_all, val_scores_all = parse_full_validation_summarization(path)

    return loss_all, steps_all, special_loss_all, special_steps_all, train_scores_all, val_loss_all, val_steps_all, \
           val_scores_all

In [ ]:
def plot_pointer_generator_performance(path: Path, dataset: str, use_full_validation: bool = False) -> None:
    train_loss, train_steps, coverage_loss, coverage_steps, _, val_loss, val_steps, scores = \
        get_model_performance(path, activate_special=lambda line: 'coverage' in line,
                              use_full_validation=use_full_validation)
    fig = plt.figure(figsize=(15, 10))
    grid = GridSpec(2, 2)
    train_ax: plt.Axes = plt.subplot(grid[0, :])
    val_ax: plt.Axes = plt.subplot(grid[1, 0])
    test_ax: plt.Axes = plt.subplot(grid[1, 1])

    train_type = ['Normal training' for _ in train_steps] + ['Training with coverage' for _ in coverage_steps]
    train_data = pd.DataFrame(
        {'Iteration': train_steps + coverage_steps, 'Loss': train_loss + coverage_loss, 'Training type': train_type}
    )
    val_data = {
        'Iteration': val_steps,
        'Loss': val_loss
    }
    test_data = pd.DataFrame(scores)
    test_data['Iteration'] = val_steps
    test_data = test_data.set_index('Iteration')

    train_ax.axvline(x=coverage_steps[0], lw=1, ls='--', c='b', alpha=0.5)
    sns.lineplot(data=train_data, x='Iteration', y='Loss', hue='Training type', ax=train_ax)
    sns.lineplot(data=val_data, x='Iteration', y='Loss', ax=val_ax, color='g')
    sns.lineplot(data=test_data, ax=test_ax)

    fig.suptitle(f'Pointer generator performance on {dataset} dataset.')
    train_ax.set_title('Loss change during training')
    val_ax.set_title('Validation loss change during training')
    test_ax.set_title('ROUGE scores change during training')

    plot_title = dataset.replace(' ', '_').replace('/', '_').lower()
    plt.savefig(f'images/results/pointer_generator_{plot_title}.png')
    plt.show()

In [ ]:
def plot_transformer_performance(path: Path, dataset: str, use_full_validation: bool = False) -> None:
    train_loss, train_steps, _, _, _, val_loss, val_steps, scores = \
        get_model_performance(path, use_full_validation=use_full_validation)
    fig = plt.figure(figsize=(15, 10))
    grid = GridSpec(2, 2)
    train_ax: plt.Axes = plt.subplot(grid[0, :])
    val_ax: plt.Axes = plt.subplot(grid[1, 0])
    test_ax: plt.Axes = plt.subplot(grid[1, 1])

    train_data = {
        'Iteration': train_steps,
        'Loss': train_loss
    }
    val_data = {
        'Iteration': val_steps,
        'Loss': val_loss
    }
    test_data = pd.DataFrame(scores)
    test_data['Iteration'] = val_steps
    test_data = test_data.set_index('Iteration')

    sns.lineplot(data=train_data, x='Iteration', y='Loss', ax=train_ax)
    sns.lineplot(data=val_data, x='Iteration', y='Loss', ax=val_ax, color='g')
    sns.lineplot(data=test_data, ax=test_ax)

    fig.suptitle(f'Transformer performance on {dataset} dataset.')
    train_ax.set_title('Loss change during training')
    val_ax.set_title('Validation loss change during training')
    test_ax.set_title('ROUGE scores change during training')

    plot_subtitle = dataset.replace(' ', '_').replace('/', '_').lower()
    plt.savefig(f'images/results/transformer_{plot_subtitle}.png')
    plt.show()

In [ ]:
def plot_rl_performance(path: Path, dataset: str, use_full_validation: bool = False) -> None:
    pretrain_loss, pretrain_steps, train_loss, train_steps, _, val_loss, val_steps, scores = \
        get_model_performance(path, activate_special=lambda line: 'pretraining' in line,
                                      use_full_validation=use_full_validation)
    fig = plt.figure(figsize=(15, 10))
    grid = GridSpec(2, 2)
    pretrain_ax: plt.Axes = plt.subplot(grid[0, 0])
    train_ax: plt.Axes = plt.subplot(grid[0, 1])
    val_ax: plt.Axes = plt.subplot(grid[1, 0])
    test_ax: plt.Axes = plt.subplot(grid[1, 1])

    pretrain_data = {
        'Iteration': pretrain_steps,
        'Loss': pretrain_loss
    }
    train_data = {
        'Iteration': train_steps,
        'Loss': train_loss
    }
    val_data = {
        'Iteration': val_steps,
        'Loss': val_loss
    }
    test_data = pd.DataFrame(scores)
    test_data['Iteration'] = val_steps
    test_data = test_data.set_index('Iteration')

    sns.lineplot(data=pretrain_data, x='Iteration', y='Loss', ax=pretrain_ax)
    sns.lineplot(data=train_data, x='Iteration', y='Loss', ax=train_ax, color='r')
    sns.lineplot(data=val_data, x='Iteration', y='Loss', ax=val_ax, color='g')
    sns.lineplot(data=test_data, ax=test_ax)

    fig.suptitle(f'RL model performance on {dataset} dataset.')
    train_ax.set_title('Loss change during pretraining')
    train_ax.set_title('Loss change during training (fine-tuning)')
    val_ax.set_title('Validation loss change during training')
    test_ax.set_title('ROUGE scores change during training')

    plot_subtitle = dataset.replace(' ', '_').replace('/', '_').lower()
    plt.savefig(f'images/results/rl_{plot_subtitle}.png')
    plt.show()

In [ ]:
def plot_ner_performance(path: Path, model: str, dataset: str) -> None:
    train_loss, train_steps, _, _, train_scores, val_loss, val_steps, val_scores = get_model_performance(path)
    fig, (loss_ax, scores_ax) = plt.subplots(2, 1, figsize=(12, 10))
    scores_ax: plt.Axes

    phases = ['Training' for _ in train_steps] + ['Validation' for _ in val_steps]
    loss_data = pd.DataFrame(
        {'Iteration': train_steps + val_steps, 'Loss': train_loss + val_loss, 'Phase': phases}
    )
    scores_data = pd.DataFrame(
        {
            'Iteration': (train_steps + val_steps) * 3,
            'Value': train_scores['F1'] + val_scores['F1'] + train_scores['Precision'] + val_scores['Precision'] +
                     train_scores['Recall'] + val_scores['Recall'],
            'Score': ['F1' for _ in train_steps + val_steps] + ['Precision' for _ in train_steps + val_steps] +
                     ['Recall' for _ in train_steps + val_steps],
            'Phase': phases * 3
        }
    )

    sns.lineplot(data=loss_data, x='Iteration', y='Loss', hue='Phase', ax=loss_ax)
    sns.lineplot(data=scores_data, x='Iteration', y='Value', hue='Score', style='Phase', ax=scores_ax)

    fig.suptitle(f'{model} performance on {dataset} dataset.')
    loss_ax.set_title('Loss change during training and validation')
    loss_ax.set_title('Scores change during training and validation')

    dataset_subtitle = dataset.replace(' ', '_').replace('/', '_').lower()
    model_subtitle = model.replace('-', '_').replace(' ', '_').lower()
    plt.savefig(f'images/results/{model_subtitle}_{dataset_subtitle}.png')
    plt.show()

In [ ]:
plot_pointer_generator_performance(logs_path / 'pointer_generator', 'CNN/Daily Mail')
plot_pointer_generator_performance(logs_path / 'pointer_generator-xsum', 'XSum', use_full_validation=True)
plot_transformer_performance(logs_path / 'transformer', 'CNN/Daily Mail')
plot_transformer_performance(logs_path / 'transformer', 'XSum')
plot_rl_performance(logs_path / 'reinforcement_learning', 'CNN/Daily Mail', use_full_validation=True)
plot_rl_performance(logs_path / 'reinforcement_learning-xsum', 'XSum', use_full_validation=True)

In [ ]:
plot_ner_performance(logs_path / 'bilstm_cnn', 'BiLSTM-CNN', 'CoNLL-2003')
plot_ner_performance(logs_path / 'bilstm_cnn-gmb', 'BiLSTM-CNN', 'GMB')
plot_ner_performance(logs_path / 'bilstm_crf', 'BiLSTM-CRF', 'CoNLL-2003')
plot_ner_performance(logs_path / 'bilstm_crf-gmb', 'BiLSTM-CRF', 'GMB')
plot_ner_performance(logs_path / 'id_cnn', 'ID-CNN', 'CoNLL-2003')
plot_ner_performance(logs_path / 'id_cnn-gmb', 'ID-CNN', 'GMB')